In [84]:
import pandas as pd
import matplotlib.pyplot as plt

# Local module
from nxr_classes import Organisations

In [6]:
nsr = pd.read_excel("nsr - alle organisasjoner.xlsx")

#### Nasjonalt skoleregister (NSR)
Udirs beskrivelse av registeret:   
"Data fra Nasjonalt skoleregister (NSR) viser enheter fra Virksomhets- og Foretaksregisteret (Vof) som tilhører grunnutdanningen. Skolene (bedriftene/virksomhetene) og deres eiere (foretak) skal være avstemt med VoF. NSR viser også Norske skoler i utlandet og inneholder noe tilleggsinformasjon utover det som fins i VoF."

Vi har hentet ut detaljert informasjon om samtlige 16 343 organisasjoner som er registrert i NSR.

I registeret er det registrert:
- Organisasjonsnummer
- Navn
- Skoleeier (kommune og fylke)
- Beliggenhet
- Kontaktinformasjon (epost, telefon og postadresse)
- Målform
- Organisasjonsform
- Næringskoder
- Skolekategori
- Skoleleder
- Elevtall
- Antall ansatte
- Skoletrinn grunnskolen (fra/til)
- Skoletrinn vgs (fra/til)
- Privatskolegodkjenning

Dette dokumentet går gjennom den mest sentrale informasjon i NSR. 

##### Aktive og nedlagte skoler

Først og fremst er det verdt å merke seg at NSR består av både aktive og nedlagte organsisasjoner. Tabellen under viser forskjellige status i registeret og hvor mange skoler som tilhører hver av dem. I NSR er det 8 332 aktive skoler, eller sagt på registerspråket: Skoler uten utgåtttype. 

In [20]:
nsr['Utgaattype_Navn'].value_counts()

Utgaattype_Navn,Ingen utgåttype,Slettet,Slettet for dublett,Slettet for sammenslåing,Utg pga eier er Enkeltpersonforetak,Utg pga eier mangler aktiv bedrift,Utg pga næringsendring
ErAktiv,,,,,,,
False,2,6093,51,416,46,709,696
True,8330,0,0,0,0,0,0


Det er også mulig å identifisere nedlagte organisasjoner med kolonnnen `ErAktiv`.

In [22]:
pd.crosstab(nsr.Utgaattype_Navn, nsr.ErAktiv)

ErAktiv,False,True
Utgaattype_Navn,,
Ingen utgåttype,2,8330
Slettet,6093,0
Slettet for dublett,51,0
Slettet for sammenslåing,416,0
Utg pga eier er Enkeltpersonforetak,46,0
Utg pga eier mangler aktiv bedrift,709,0
Utg pga næringsendring,696,0


##### Identifisere skolene
Det er ikke alle organisasjoner i NSR som er en skole. 9 540 av organisasjonene er registrert som skole, og samtlige av dem står oppført med organisasjonsformen `Bedrift`. Organisasjoner registrert i NSR som ikke er skoler er i ofte skoleeiere.  

In [25]:
pd.crosstab(nsr.Organisasjonsform_Navn, nsr.ErSkole)

ErSkole,False,True
Organisasjonsform_Navn,,
Aksjeselskap,2485,0
Annen juridisk person,34,0
Annet foretak iflg. særskilt lov,18,0
Ansvarlig selskap,50,0
Ansvarlig selskap med delt deltakeransvar,145,0
Bedrift,647,9540
Den norske kirke,2,0
Enkeltpersonforetak,456,0
Forening/lag/innretning,603,0


In [64]:
pd.crosstab(nsr.ErSkole, nsr.ErSkoleeier)

ErSkoleeier,False,True
ErSkole,,
False,2876,3927
True,9540,0


Fra nå av fokuserer vi bare på organisasjonene registrert som aktive skoler. Det er 5 566 organisasjoner som er registrert som aktive skoler.  

In [45]:
pd.crosstab(nsr.ErSkole, nsr.ErAktiv)

ErAktiv,False,True
ErSkole,,
False,4039,2764
True,3974,5566


In [46]:
skoler = nsr.query('ErAktiv == True & ErSkole == True')

##### Skolekategori

Skolene i NSR er registrert med forskjellige kategorier. Brukte kategorier, er: 

- Offentlig skole
- Privatskole
- Grunnskole
- Videregående skole
- Spesialskole
- Grunnopplæring

Som det fremgår av kategoriene kan én skole være registrert med flere skolekategorier. Når det er sagt er noen av kategoriene gjensidig utelukkende, slik som privat skole og offentligskole. Det er minimalt med overlapp mellom kategoriene grunnskole og videregående opplæring, men det er 42 skoler som er registrert i begge kategorier. 

Alle grunnskoler og videregående skoler er er naturlig definert med kategorien grunnopplæring.

In [47]:
pd.crosstab(skoler.ErVideregaaendeSkole, skoler.ErGrunnskole)

ErGrunnskole,False,True
ErVideregaaendeSkole,,
False,2258,2760
True,506,42


In [53]:
pd.crosstab(skoler.ErGrunnopplaering, skoler.ErGrunnskole)

ErGrunnskole,False,True
ErGrunnopplaering,,
False,2204,0
True,560,2802


In [54]:
pd.crosstab(skoler.ErGrunnopplaering, skoler.ErVideregaaendeSkole)

ErVideregaaendeSkole,False,True
ErGrunnopplaering,,
False,2204,0
True,2814,548


In [62]:
ikke_grunnopp = skoler.query('ErGrunnopplaering == False')
ikke_grunnopp.shape

(2204, 86)

In [74]:
pd.crosstab(ikke_grunnopp.ErOffentligSkole, ikke_grunnopp.Skolekategorier_Navn)
#ikke_grunnopp['ErPrivatskole'].value_counts()

Skolekategorier_Navn,Offentlig skole,Privat eiet
ErOffentligSkole,,
False,0,1338
True,866,0


In [90]:
def walk_dict(d, new_dict, parent_key = ''):

        for key, val in d.items():
            # If val is a dictionary, traverse
            # the nested dictionary
            if isinstance(val, dict):
                walk_dict(val, new_dict, parent_key = key + '_')
            elif isinstance(val, list) and len(val) > 0:
                print(type(val[0]))
                walk_dict(val[0], new_dict, parent_key = key + '_')
            else:
                print(key, ":", val)
                new_dict[parent_key + key] = val

        return new_dict

organisations = Organisations('nsr')

organisations.get_org('998763509')

walk_dict(organisations.json['998763509'], dict())



Orgnr : 998763509
Navn : Hå opplæringssenter
Karakteristikk : Avd voksenopplæring
Fylkesnr : 11
Navn : Rogaland
Orgnr : 971045698
OrgnrStatsforvalter : 974763230
Kommunenr : 1119
Navn : Hå
Orgnr : 964969590
ErNedlagt : False
Fylkesnr : 11
Gruppe : 7
Navn : KOSTRA-gruppe 7
Adresse : Jadarvegen 5
Postnr : 4365
Poststed : NÆRBØ
Land : Norge
Adresse : Postboks 33
Postnr : 4367
Poststed : NÆRBØ
Land : Norge
Lengdegrad : 5.63935
Breddegrad : 58.66529
Zoom : 12
GeoKilde : GeoNorge
Epost : post.opplaringssenter@ha.kommune.no
Url : None
Id : B
Navn : Bokmål
Telefon : 51 43 60 90
Mobil : None
Fax : None
Id : BEDR
Navn : Bedrift
Prioritet : 1
ErHjelpeenhetskode : False
Kode : 85.594
Navn : Voksenopplæringssentre
Versjon : 2007
Id : A
Navn : Ingen utgåttype
ErAktiv : True
ErSkole : True
ErSkoleeier : False
ErGrunnskole : False
ErPrivatskole : False
ErOffentligSkole : True
ErVideregaaendeSkole : False
ErSpesialskole : False
ErGrunnopplaering : False
PrivatskoleGodkjenning : None
Fornavn : 
Etternav

{'Orgnr': '998763509',
 'Navn': 'Hå opplæringssenter',
 'Karakteristikk': 'Avd voksenopplæring',
 'Fylke_Fylkesnr': '11',
 'Fylke_Navn': 'Rogaland',
 'Fylke_Orgnr': '971045698',
 'Fylke_OrgnrStatsforvalter': '974763230',
 'Kommune_Kommunenr': '1119',
 'Kommune_Navn': 'Hå',
 'Kommune_Orgnr': '964969590',
 'Kommune_ErNedlagt': False,
 'Kommune_Fylkesnr': '11',
 'Kommunegruppe_Gruppe': 7,
 'Kommunegruppe_Navn': 'KOSTRA-gruppe 7',
 'Beliggenhetsadresse_Adresse': 'Jadarvegen 5',
 'Beliggenhetsadresse_Postnr': '4365',
 'Beliggenhetsadresse_Poststed': 'NÆRBØ',
 'Beliggenhetsadresse_Land': 'Norge',
 'Postadresse_Adresse': 'Postboks 33',
 'Postadresse_Postnr': '4367',
 'Postadresse_Poststed': 'NÆRBØ',
 'Postadresse_Land': 'Norge',
 'Koordinat_Lengdegrad': 5.63935,
 'Koordinat_Breddegrad': 58.66529,
 'Koordinat_Zoom': 12,
 'Koordinat_GeoKilde': 'GeoNorge',
 'Epost': 'post.opplaringssenter@ha.kommune.no',
 'Url': None,
 'Maalform_Id': 'B',
 'Maalform_Navn': 'Bokmål',
 'Telefon': '51 43 60 90',
 '